# Technical Analysis


## Package: ta
- [Technical Analysis Library in Python](https://github.com/bukosabino/ta)
    - Official document: https://technical-analysis-library-in-python.readthedocs.io/en/latest/
    - Another example: https://github.com/bukosabino/ta/blob/master/examples_to_use/visualize_features.ipynb
- References:
    - https://school.stockcharts.com/doku.php?id=technical_indicators
    - https://www.tradingtechnologies.com/help/x-study/technical-indicator-definitions/list-of-technical-indicators/
    - https://www.bernstein-bank.com/wp-content/uploads/2016/08/Technical-Analysis-Englisch.pdf

In [ ]:
%%capture

!pip install --upgrade pandas-datareader
!pip install ffn ta

In [ ]:
import ffn
import pandas as pd
import ta
import matplotlib.pyplot as plt

params = {'legend.fontsize': 'x-large',
      'figure.figsize': (12, 6),
      'axes.labelsize': 'x-large',
      'axes.titlesize': 'x-large',
      'xtick.labelsize': 'x-large',
      'ytick.labelsize': 'x-large'}
plt.rcParams.update(params)

%matplotlib inline

In [ ]:
df = ffn.get("2330.tw", start = "2020-01-01")

df

### Example 1: Simple Moving Average


In [ ]:
from ta.trend import SMAIndicator

%matplotlib inline

df["SMA5"] = SMAIndicator(df["2330tw"], window = 5).sma_indicator()
df["SMA10"] = SMAIndicator(df["2330tw"], window = 10).sma_indicator()
df["SMA20"] = SMAIndicator(df["2330tw"], window = 20).sma_indicator()
df["SMA60"] = SMAIndicator(df["2330tw"], window = 60).sma_indicator()
df["SMA120"] = SMAIndicator(df["2330tw"], window = 120).sma_indicator()
df["SMA250"] = SMAIndicator(df["2330tw"], window = 250).sma_indicator()

ax = df["2330tw"].plot.line(figsize = (12, 6), ylabel = "Price")
df.iloc[:, 1:].plot.line(ax = ax, linestyle = "--", grid = True, title = "2330tw")

### Example 2: Bollinger Bands
- John Bollinger started developing Bollinger Bands in the early 1980s.
    - https://www.bollingerbands.com/
- Trading signals:
    - Bollinger bands often represent strong support lines or resistance lines.
    - Extreme highs and lows outside the bands indicate a possible trend reversal.
    - Narrowing of the ligaments can in turn indicate a possible impending breakout.
    - If the price touches, exceeds or falls below a band several times in succession, a trend acceleration in the existing direction is expected. 
- References:
    - https://rich01.com/what-is-bollinger-band/

In [ ]:
from ta.volatility import BollingerBands

bb_indicator = BollingerBands(close = df["2330tw"], window = 20, window_dev = 2)
df["bbm"] = bb_indicator.bollinger_mavg()
df["bbh"] = bb_indicator.bollinger_hband()
df["bbl"] = bb_indicator.bollinger_lband()

In [ ]:
ax = df["2330tw"].plot.line(figsize = (12, 6), ylabel = "Price")
ax = df[["bbm", "bbh", "bbl"]].plot.line(ax = ax, linestyle = "--", grid = True, title = "2330tw")

In [ ]:
ax = df["2330tw"].plot.line(figsize = (12, 6), ylabel = "Price")
ax = df[["bbm", "bbh", "bbl"]].plot.line(ax = ax, linestyle = "--", grid = True, title = "2330tw")

bbh_mask = bb_indicator.bollinger_hband_indicator() == 1
bbl_mask = bb_indicator.bollinger_lband_indicator() == 1

plt.plot(df["2330tw"][bbh_mask], "ro")
plt.plot(df["2330tw"][bbl_mask], "go")

### Example 3: Moving Average Convergence Divergence (MACD)
- Gerald Appel (1979).
- Exponential Moving Average (EMA)
    - Let $N$ be the number of observation days and $\alpha = \frac{2}{N + 1}$.
    - Then we have $$\text{EMA}' = \alpha \times \text{close}_i + (1 - \alpha) \times \text{EMA}.$$
    - EMAs reduce the lag **by applying more weight to recent prices**.

<center>
<img src = "https://fxcodebase.com/wiki/images/0/07/Ema1.png" width = "500px"/><br>
Figure: $N = 10$
</center>

- Calculation:
    - MACD Line: 12-day EMA $-$ 26-day EMA.
    - Signal Line: 9-day EMA of MACD Line.
    - MACD Histogram: MACD Line $-$ Signal Line.
- References:
    - https://www.taindicators.com/macd/
    - https://school.stockcharts.com/doku.php?id=technical_indicators:moving_average_convergence_divergence_macd

In [ ]:
from ta.trend import macd_diff

df["macd_diff"] = macd_diff(close = df["2330tw"],  window_sign = 9, window_fast = 12, window_slow = 26)

In [ ]:
plt.figure(figsize = (12, 6))
plt.subplot(211)
plt.title("2330tw")
plt.plot(df.index, df["2330tw"], "o", linewidth = 0.5, linestyle = "--", markersize = 2)
plt.grid(True)
plt.ylabel("Price")
l, h = plt.xlim()

plt.subplot(212)
plt.bar(df.index[df["macd_diff"] > 0], df["macd_diff"][df["macd_diff"] > 0], color = "r")
plt.bar(df.index[df["macd_diff"] <= 0], df["macd_diff"][df["macd_diff"] <= 0], color = "g")
plt.grid(True)
plt.ylabel("MACD")
plt.xlim(l, h)

plt.show()

### Example 4: Stochastic Oscillator (KD)
- George C. Lan (1957).
- The Stochastic Oscillator is a **momentum** indicator that shows the location of the close relative to the high-low range over a set number of periods.
- Calculation:
    - Lowest Low = lowest low for the look-back period
    - Highest High = highest high for the look-back period
    - %K = (Current Close - Lowest Low) / (Highest High - Lowest Low) x 100
    - %D = 3-day SMA of %K

    
    

In [ ]:
from ta.momentum import StochasticOscillator

df2 = ffn.get("2330.tw:High, 2330.tw:Low, 2330.tw:Close", start = "2020-01-01")
df2.columns = ["High", "Low", "Close"]

In [ ]:
df2["K"] = StochasticOscillator(close = df2["Close"], low = df2["Low"], high = df2["High"], window = 14, smooth_window = 3).stoch()
df2["D"] = StochasticOscillator(close = df2["Close"], low = df2["Low"], high = df2["High"], window = 14, smooth_window = 3).stoch_signal()

In [ ]:
plt.figure(figsize = (12, 6))
plt.subplot(211)
plt.title("2330")
plt.plot(df2.index, df2["Close"], "o", linewidth = 0.5, linestyle = "--", markersize = 2)
plt.grid(True)
plt.ylabel("Price")
l, h = plt.xlim()

plt.subplot(212)
plt.plot(df2.index, df2["K"], color = "r")
plt.plot(df2.index, df2["D"], color = "g")
plt.grid(True)
plt.ylabel("Stochastic Oscillator (%)")
plt.legend(["K (14)", "D (3)"])
plt.xlim(l, h)

plt.show()

### Example 5: Relative Strength Index (RSI)
- Wells Wilder (1978).
- Calculation:
    RSI = 100 - 100 / (1 + RS) with RS = Average Gain / Average Loss.
- RSI is more sensitive to price change than KD.
- References:
    - https://rich01.com/rsi-index-review/

In [ ]:
from ta.momentum import RSIIndicator

df["RSI"] = RSIIndicator(close = df["2330tw"], window = 14).rsi()

In [ ]:
plt.figure(figsize = (12, 6))
plt.subplot(211)
plt.title("2330")
plt.plot(df.index, df["2330tw"], "o", linewidth = 0.5, linestyle = "--", markersize = 2)
plt.grid(True)
plt.ylabel("Price")
l, h = plt.xlim()

plt.subplot(212)
plt.plot(df.index, df["RSI"], color = "r")
plt.grid(True)
plt.ylabel("RSI (%)")
plt.xlim(l, h)

plt.show()